##### I recommend you to review this notebook via Google Colab so that you don't drown in long code blocks.
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1LxhnjaXNNIOJNGhSfY3wkGSAdSZUaaAC?usp=sharing)


# INTRO
**Hello,**

This notebook is created for the Doğuş Datathon Competition by **Yasin İnal and Selim Yürekli.**

The goal was to predict which users will open a sales file in a specified time interval (june-july 2021) based on a raw data that is extracted from a **relational database.**
[Inspect the data here.](https://www.kaggle.com/c/dogus-datathon-otomotiv/data)

To accomplish this, me and my friend has implemented various data cleaning techniques and tried different algorithms like ***Deep Learning, LightGBM, XGBoost*** to train the model. [The LightGBM was the best performer. But we only listed Deep Learning here we used LightGBM in a different notebook to speed up training]

The most challenging parts of the job was **to understand the relation between the database tables, merging them by calculating time-specific sale informations of the customers in a specified time interval and filling the columns that has a huge NA value ratio (more than 40%) with specific algorithms.**

### Data Cleaning:

To fill the NA values, other than using classical data cleaning techniques like filling the NA values with median or mean values, we have filled those values **with a trick** as described below:

 - We noticed that some customers had multiple entries in the database.
 - And despite they left NA values in some forms, they might be filled  
   that values in another form.
 - Starting from here, **we detected the customers that have multiple
   entries and filled their NA values if they had filled that part
   at least one time in another entry**.
  - Even though we have used vectorization, the time complexity of the algorithm was ***O(n^2),*** and it was taking more than 30 minutes to execute this cleaning process in a dataset that has more than ***500k rows.***

**To merge all the data from different tables, we have written our own algorithms.** For example, to create the final table that we have used in the  training, we calculated how many sales files a customer had opened in the past 3 months, past 1 year and 5 year and added them as x values for every customer. For y (output) values, we calculated if that user opened a sales file at least one time in a specific time interval (march-may 2021 for us, we could have tried different time intervals for training but our time was limited). So that, we could understand if a user will open a sales file based on his/her past sales data.

### Training
As we have mentioned above, we tried LightGBM, XGBoost and Deep Learning to train model. Surprisingly, the LightGBM was the best performer even though we have had enough neurons in our DL model and used regularization techniques like Dropout and BatchNormalization.
 

In [ ]:
!shutdown

System has not been booted with systemd as init system (PID 1). Can't operate.


In [ ]:
!rm -d /root/.kaggle

rm: cannot remove '/root/.kaggle': No such file or directory


In [ ]:
!mkdir /root/.kaggle

In [ ]:
!cp ./drive/MyDrive/kaggle.json /root/.kaggle 

In [ ]:
!kaggle competitions download -c dogus-datathon-otomotiv

  0% 0.00/7.75M [00:00<?, ?B/s]
100% 7.75M/7.75M [00:00<00:00, 128MB/s]
 48% 6.00M/12.6M [00:00<00:00, 60.2MB/s]
100% 12.6M/12.6M [00:00<00:00, 80.3MB/s]
  0% 0.00/5.11M [00:00<?, ?B/s]
100% 5.11M/5.11M [00:00<00:00, 84.5MB/s]
  0% 0.00/2.69M [00:00<?, ?B/s]
100% 2.69M/2.69M [00:00<00:00, 86.4MB/s]
 96% 12.0M/12.5M [00:00<00:00, 124MB/s]
100% 12.5M/12.5M [00:00<00:00, 115MB/s]
  0% 0.00/4.71M [00:00<?, ?B/s]
100% 4.71M/4.71M [00:00<00:00, 155MB/s]
  0% 0.00/525k [00:00<?, ?B/s]
100% 525k/525k [00:00<00:00, 91.3MB/s]


# Intro

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

F_CUST_DATA = pd.read_csv("FINAL_CUSTOMER_DATATHON.csv.zip") 
F_CUST_RELATED_DATA = pd.read_csv("FINAL_CUSTOMER_RELATED_TABLE_FOR_DATATHON.csv.zip") 
F_SALES_DATA = pd.read_csv("FINAL_SALES_FILE_DATATHON.csv.zip") 
F_SIFIR_ARAC_DATA = pd.read_csv("FINAL_SIFIR_ARAC_ALANLAR_DATATHON.csv.zip")
F_VEHICLE_DATA = pd.read_csv("FINAL_VEHICLE_TABLE_DATATHON.csv.zip") 
MASK_SERVIS_DATA = pd.read_csv("MASK_SERVIS_BAKIM_DATATHON_FINAL.csv.zip") 
ALL_DATAS = np.array([F_CUST_DATA,F_CUST_RELATED_DATA,F_SALES_DATA,F_SIFIR_ARAC_DATA,F_VEHICLE_DATA,MASK_SERVIS_DATA])
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
current_year = 2021

In [ ]:
# sifir_arac_data = pd.merge(F_SIFIR_ARAC_DATA,F_VEHICLE_DATA,on = 'VEHICLE_ID')
# common = F_VEHICLE_DATA.index.intersection(F_SIFIR_ARAC_DATA.VEHICLE_ID)

# Null Value Check

In [ ]:
F_CUST_DATA.isna().sum()

Unnamed: 0                            0
BASE_CUSTOMER_ID                      0
CUSTOMER_ID                           0
GENDER                                3
GENDER_ID                             0
MARITAL_STATUS                   229117
MARITAL_STATUS_ID                     0
BIRTH_DATE                       204472
FK_ADDRESS_COMMUNICATION_CITY    119578
OCCUPATION                       468867
dtype: int64

In [ ]:
F_CUST_RELATED_DATA.isna().sum()

Unnamed: 0                             0
BASE_CUSTOMER_ID                       0
VEHICLE_ID                             0
START_DATE                             0
END_DATE                          215122
FK_RELATION_STATUS_ID                  0
FK_RELATION_STATUS_EXPLANATION         0
dtype: int64

In [ ]:
F_SALES_DATA.isna().sum()

Unnamed: 0             0
CUSTOMER_ID            0
SALESFILE_ID           0
SF_CREATE_DATE         0
STATUS                 0
REQ_BRAND_CODE         0
REQ_TOPMODEL_CODE    289
dtype: int64

In [ ]:
F_SIFIR_ARAC_DATA.isna().sum()

Unnamed: 0     0
VEHICLE_ID     0
CUSTOMER_ID    0
CREATE_DATE    0
dtype: int64

In [ ]:
F_VEHICLE_DATA.isna().sum()

Unnamed: 0            0
VEHICLE_ID            0
TRAFFIC_DATE        866
BRAND_CODE            0
BASEMODEL_CODE        0
TOPMODEL_CODE         0
MOTOR_GAS_TYPE    72446
GEAR_BOX_TYPE     84015
dtype: int64

In [ ]:
F_VEHICLE_DATA.shape[0]

345092

# Dealing With Null Values

### fill_the_table_with_baseid

In [ ]:

def fill_the_table_with_baseid(df,base_column, report_in_every=10000):
  filled_count = 0 
  count = 0

  for column in df: # iterate through columns
    count=0
    if column == base_column: # skip the base column
      continue
    df_null = df[ df[column].isna() ] # take the null values of our column

    for index in df_null.index: # iterate through null values of our column
      # report the filled row percentage in every 10,000 rows.
      if count % report_in_every == 0: 
        filled_percentage = (count/df_null.shape[0]) * 100
        print(column, filled_percentage)
      BASE_ID = df.loc[index, base_column]
      df_BASEID = df[ df[base_column]==BASE_ID ] 

      df_non_empty = df_BASEID[ df_BASEID[column].notna() ]
      #get non na values of a specific baseid
      if df_non_empty.shape[0] > 0: # if baseid include non na value then::
        df.at[index, column] = df_non_empty[column].iloc[0] #
        filled_count += 1
      count += 1
    print(column, 100.0)
  print('Filled', filled_count, 'values in total.')
  return df

# Customer DF Operations

In [ ]:
df_customer = F_CUST_DATA.copy()

In [ ]:
df_customer = df_customer.drop('MARITAL_STATUS_ID', axis=1)
df_customer = df_customer.drop('GENDER_ID', axis=1)

In [ ]:
df_customer.isna().sum()

Unnamed: 0                            0
BASE_CUSTOMER_ID                      0
CUSTOMER_ID                           0
GENDER                                3
MARITAL_STATUS                   229117
BIRTH_DATE                       204472
FK_ADDRESS_COMMUNICATION_CITY    119578
OCCUPATION                       468867
dtype: int64

In [ ]:
df_customer.dtypes

Unnamed: 0                         int64
BASE_CUSTOMER_ID                   int64
CUSTOMER_ID                        int64
GENDER                            object
MARITAL_STATUS                    object
BIRTH_DATE                       float64
FK_ADDRESS_COMMUNICATION_CITY     object
OCCUPATION                        object
dtype: object

In [ ]:
df_customer.dropna().agg([min, max])

,Unnamed: 0,BASE_CUSTOMER_ID,CUSTOMER_ID,GENDER,MARITAL_STATUS,BIRTH_DATE,FK_ADDRESS_COMMUNICATION_CITY,OCCUPATION
min,1,3,1000003,Erkek,Bekar,1914.0,Adana,Akademisyen / Öğr.görevlisi / Araştırma Görevlisi
max,849918,196556,1849873,Kadın,Evli,2020.0,Şırnak,Şöför / Taksici / Transfer


In [ ]:
df_customer['BASE_CUSTOMER_ID'] = df_customer['BASE_CUSTOMER_ID'].astype('uint32')
df_customer['CUSTOMER_ID'] = df_customer['CUSTOMER_ID'].astype('uint32')
df_customer['Unnamed: 0'] = df_customer['Unnamed: 0'].astype('uint32')
df_customer.dtypes

Unnamed: 0                        uint32
BASE_CUSTOMER_ID                  uint32
CUSTOMER_ID                       uint32
GENDER                            object
MARITAL_STATUS                    object
BIRTH_DATE                       float64
FK_ADDRESS_COMMUNICATION_CITY     object
OCCUPATION                        object
dtype: object

## CUST_DATA Filling The Customer Values

### Old Code

In [ ]:
# # count = 0
# # filled_count = 0

# for column in df:
#   # count=0
#   if column == 'BASE_CUSTOMER_ID':
#     continue
#   df_null = df[ df[column].isna() ]
#   for index in df_null.index:
#     # if count % 50000 == 0:
#     #   print(column, count)
#     base_customer = df.loc[index, 'BASE_CUSTOMER_ID']
#     df_customer = df[ df['BASE_CUSTOMER_ID']==base_customer ] 
#     for row_index, row in df_customer.iterrows():
#       if not pd.isna(row)[column]:
#         new_value = row[column]
#         df.at[index, column] = new_value
#         # print(column, index, new_value)
#         # filled_count += 1
#         break
#     # count += 1
# # print(filled_count)

MARITAL_STATUS 0
MARITAL_STATUS 50000
MARITAL_STATUS 100000
MARITAL_STATUS 150000
MARITAL_STATUS 200000
BIRTH_DATE 0
BIRTH_DATE 50000
BIRTH_DATE 100000
BIRTH_DATE 150000
BIRTH_DATE 200000
FK_ADDRESS_COMMUNICATION_CITY 0
FK_ADDRESS_COMMUNICATION_CITY 50000
FK_ADDRESS_COMMUNICATION_CITY 100000
OCCUPATION 0
OCCUPATION 50000
OCCUPATION 100000
OCCUPATION 150000
OCCUPATION 200000
OCCUPATION 250000
OCCUPATION 300000
OCCUPATION 350000
OCCUPATION 400000
OCCUPATION 450000
638398


### New Code

In [ ]:
fill_the_table_with_baseid(df_customer, 'BASE_CUSTOMER_ID', report_in_every=80000)

Unnamed: 0 100.0
CUSTOMER_ID 100.0
GENDER 0.0
GENDER 100.0
MARITAL_STATUS 0.0
MARITAL_STATUS 91.41498977294802
MARITAL_STATUS 100.0
BIRTH_DATE 0.0
BIRTH_DATE 39.12516139129074
BIRTH_DATE 78.25032278258148
BIRTH_DATE 100.0
FK_ADDRESS_COMMUNICATION_CITY 0.0
FK_ADDRESS_COMMUNICATION_CITY 66.90193848366756
FK_ADDRESS_COMMUNICATION_CITY 100.0
OCCUPATION 0.0
OCCUPATION 17.062407889657408
OCCUPATION 34.124815779314815
OCCUPATION 51.18722366897222
OCCUPATION 68.24963155862963
OCCUPATION 85.31203944828704
OCCUPATION 100.0
Filled 496794 values in total.


,Unnamed: 0,BASE_CUSTOMER_ID,CUSTOMER_ID,GENDER,MARITAL_STATUS,BIRTH_DATE,FK_ADDRESS_COMMUNICATION_CITY,OCCUPATION
0,0,158891,1747700,Erkek,Evli,1975.0,İstanbul,Diğer
1,1,169233,1710561,Erkek,Evli,1962.0,Erzurum,Öğretmen / Eğitmen
2,2,30887,1043923,Erkek,Evli,1971.0,İstanbul,Esnaf
3,3,38013,1101926,Erkek,NaN,NaN,Manisa,NaN
4,4,157091,1819787,Kadın,Evli,1949.0,Ankara,Öğretmen / Eğitmen
...,...,...,...,...,...,...,...,...
849914,849914,18175,1744051,Erkek,Evli,1984.0,Gaziantep,Tekstilci
849915,849915,108604,1209486,Erkek,Evli,1969.0,İstanbul,Nakliyeci / Lojistik / Taşımacılık
849916,849916,155926,1670812,Erkek,Evli,1970.0,İstanbul,NaN
849917,849917,123223,1691689,Erkek,Evli,1970.0,Kayseri,NaN


In [ ]:
df_customer.to_csv('df_customer_filled_v2.csv')

In [ ]:
!cp df_customer_filled_v2.csv drive/MyDrive/

In [ ]:
df_customer = pd.read_csv('drive/MyDrive/df_customer_filled_v2.csv')

## Customer Drop City

In [ ]:
df_customer = df_customer.drop('FK_ADDRESS_COMMUNICATION_CITY', axis=1)

In [ ]:
df_customer.head(1)

,Unnamed: 0,BASE_CUSTOMER_ID,CUSTOMER_ID,GENDER,MARITAL_STATUS,BIRTH_DATE,OCCUPATION
0,0,158891,1747700,Erkek,Evli,1975.0,Diğer


## Customer BIRTH_DATE to AGE

In [ ]:
df_customer['AGE'] = df_customer.BIRTH_DATE.map(lambda date: current_year-date)
df_customer.head(5)

,Unnamed: 0,BASE_CUSTOMER_ID,CUSTOMER_ID,GENDER,MARITAL_STATUS,BIRTH_DATE,OCCUPATION,AGE
0,0,158891,1747700,Erkek,Evli,1975.0,Diğer,46.0
1,1,169233,1710561,Erkek,Evli,1962.0,Öğretmen / Eğitmen,59.0
2,2,30887,1043923,Erkek,Evli,1971.0,Esnaf,50.0
3,3,38013,1101926,Erkek,NaN,NaN,NaN,NaN
4,4,157091,1819787,Kadın,Evli,1949.0,Öğretmen / Eğitmen,72.0


In [ ]:
df_customer = df_customer.drop('BIRTH_DATE', axis=1)
df_customer.head(1)

,Unnamed: 0,BASE_CUSTOMER_ID,CUSTOMER_ID,GENDER,MARITAL_STATUS,OCCUPATION,AGE
0,0,158891,1747700,Erkek,Evli,Diğer,46.0


In [ ]:
df_customer.drop(df_customer.columns[0], axis=1).to_csv('drive/MyDrive/df_customer_filled_v2.csv')

In [ ]:
df_customer.head(1)

,Unnamed: 0,BASE_CUSTOMER_ID,CUSTOMER_ID,GENDER,MARITAL_STATUS,OCCUPATION,AGE
0,0,158891,1747700,Erkek,Evli,Diğer,46.0


# VEHICLE

### Vehicle Typo Correct

In [ ]:
typo_gear = np.array([
    ('Otomatik (DSG)', 'OTOMATİK'),
    ('Otomatik', 'OTOMATİK'),
    ('Mekanik', 'MANUEL'),
    ('Sürekli Değişken', 'OTOMATİK'),
    ('Otomaik', 'OTOMATİK'),
    ('?automatisch', 'OTOMATİK'),
    ('Otomatik Değişken', 'OTOMATİK'),
    ('Sürekli değişken', 'OTOMATİK'),
    ('Otomatik (DSG Şanzuman)', 'OTOMATİK'),
    ('Düz', 'MANUEL'),
    ('Otomatik(DSG)', 'OTOMATİK'),
    ('?Handschaltung', 'MANUEL'),
    ('manuel', 'MANUEL'),
    ('Manuel', 'MANUEL')
])
typo_gas_type = np.array([
    ('Diesel', 'Dizel'),
    ('Kurşunsuz benzin', 'Benzin'),
    ('Kurşunsuz Benzin', 'Benzin'),
    ('Seçiniz', None),
    ('?Benzin', 'Benzin'),
    ('Benzin/Hybrid', 'Benzin/Hibrit'),
])

def typo_regulate(typo,column_name,data):
    for key,val in typo:
        data[column_name].replace(key,val,inplace=True)

df_vehicle = F_VEHICLE_DATA.copy()
typo_regulate(typo_gear,'GEAR_BOX_TYPE', df_vehicle)
typo_regulate(typo_gas_type,'MOTOR_GAS_TYPE', df_vehicle)

In [ ]:
df_vehicle['GEAR_BOX_TYPE'].unique()

array(['MANUEL', 'OTOMATİK', nan], dtype=object)

In [ ]:
df_vehicle.isna().sum()

Unnamed: 0            0
VEHICLE_ID            0
TRAFFIC_DATE        866
BRAND_CODE            0
BASEMODEL_CODE        0
TOPMODEL_CODE         0
MOTOR_GAS_TYPE    73526
GEAR_BOX_TYPE     84015
dtype: int64

### Vehicle Dtype Casting


In [ ]:
df_vehicle.dtypes

Unnamed: 0         int64
VEHICLE_ID         int64
TRAFFIC_DATE      object
BRAND_CODE        object
BASEMODEL_CODE     int64
TOPMODEL_CODE      int64
MOTOR_GAS_TYPE    object
GEAR_BOX_TYPE     object
dtype: object

In [ ]:
df_vehicle.agg([min,max])

,Unnamed: 0,VEHICLE_ID,BRAND_CODE,BASEMODEL_CODE,TOPMODEL_CODE
min,0,5000001,K,10706,9083
max,345091,5345092,Z,18140,10010


In [ ]:
df_vehicle['BASEMODEL_CODE'] =  df_vehicle['BASEMODEL_CODE'].astype('uint16')
df_vehicle['TOPMODEL_CODE'] = df_vehicle['TOPMODEL_CODE'].astype('uint16')
df_vehicle['VEHICLE_ID'] = df_vehicle['VEHICLE_ID'].astype('uint32')
df_vehicle['Unnamed: 0'] = df_vehicle['VEHICLE_ID'].astype('uint32')
df_vehicle.dtypes

Unnamed: 0        uint32
VEHICLE_ID        uint32
TRAFFIC_DATE      object
BRAND_CODE        object
BASEMODEL_CODE    uint16
TOPMODEL_CODE     uint16
MOTOR_GAS_TYPE    object
GEAR_BOX_TYPE     object
dtype: object

In [ ]:
F_VEHICLE_DATA

,Unnamed: 0,VEHICLE_ID,TRAFFIC_DATE,BRAND_CODE,BASEMODEL_CODE,TOPMODEL_CODE,MOTOR_GAS_TYPE,GEAR_BOX_TYPE
0,0,5317713,2017-12-07,Z,13398,9349,Benzin,MANUEL
1,1,5259212,2014-12-09,X,15694,9524,Dizel,OTOMATİK
2,2,5110414,2005-12-12,Y,11183,9118,Benzin,OTOMATİK
3,3,5101263,2005-08-23,X,15039,9495,Benzin,MANUEL
4,4,5053909,2003-04-09,Y,12255,9219,NaN,NaN
...,...,...,...,...,...,...,...,...
345087,345087,5253033,2014-07-24,T,13515,9352,Benzin,MANUEL
345088,345088,5174679,2010-05-27,T,13833,9376,Benzin,MANUEL
345089,345089,5059846,2004-12-14,X,15316,9509,Benzin,OTOMATİK
345090,345090,5080882,2004-06-29,K,14615,9446,Dizel,MANUEL


### VEHICLE_DATA Filling

In [ ]:
fill_the_table_with_baseid(df_vehicle, 'TOPMODEL_CODE')

Unnamed: 0 100.0
VEHICLE_ID 100.0
TRAFFIC_DATE 0.0
TRAFFIC_DATE 100.0
BRAND_CODE 100.0
BASEMODEL_CODE 100.0
MOTOR_GAS_TYPE 0.0
MOTOR_GAS_TYPE 13.600631069281615
MOTOR_GAS_TYPE 27.20126213856323
MOTOR_GAS_TYPE 40.80189320784484
MOTOR_GAS_TYPE 54.40252427712646
MOTOR_GAS_TYPE 68.00315534640808
MOTOR_GAS_TYPE 81.60378641568968
MOTOR_GAS_TYPE 95.2044174849713
MOTOR_GAS_TYPE 100.0
GEAR_BOX_TYPE 0.0
GEAR_BOX_TYPE 11.902636433970125
GEAR_BOX_TYPE 23.80527286794025
GEAR_BOX_TYPE 35.70790930191038
GEAR_BOX_TYPE 47.6105457358805
GEAR_BOX_TYPE 59.51318216985062
GEAR_BOX_TYPE 71.41581860382075
GEAR_BOX_TYPE 83.31845503779087
GEAR_BOX_TYPE 95.221091471761
GEAR_BOX_TYPE 100.0
Filled 64576 values in total.


,Unnamed: 0,VEHICLE_ID,TRAFFIC_DATE,BRAND_CODE,BASEMODEL_CODE,TOPMODEL_CODE,MOTOR_GAS_TYPE,GEAR_BOX_TYPE
0,5317713,5317713,2017-12-07,Z,13398,9349,Benzin,MANUEL
1,5259212,5259212,2014-12-09,X,15694,9524,Dizel,OTOMATİK
2,5110414,5110414,2005-12-12,Y,11183,9118,Benzin,OTOMATİK
3,5101263,5101263,2005-08-23,X,15039,9495,Benzin,MANUEL
4,5053909,5053909,2003-04-09,Y,12255,9219,Dizel,OTOMATİK
...,...,...,...,...,...,...,...,...
345087,5253033,5253033,2014-07-24,T,13515,9352,Benzin,MANUEL
345088,5174679,5174679,2010-05-27,T,13833,9376,Benzin,MANUEL
345089,5059846,5059846,2004-12-14,X,15316,9509,Benzin,OTOMATİK
345090,5080882,5080882,2004-06-29,K,14615,9446,Dizel,MANUEL


In [ ]:
# df_vehicle.to_csv('df_vehicle_filled.csv')
# !cp df_vehicle_filled.csv drive/MyDrive/

In [ ]:
# df_vehicle = pd.read_csv('drive/MyDrive/df_vehicle_filled.csv')

## VEHICLE_DATA DateTime Conversion

In [ ]:
# df_vehicle

In [ ]:
df_vehicle.isna().mean() * 100 # Get Null Values as ''Percentage''

Unnamed: 0         0.000000
Unnamed: 0.1       0.000000
VEHICLE_ID         0.000000
TRAFFIC_DATE       0.000000
BRAND_CODE         0.000000
BASEMODEL_CODE     0.000000
TOPMODEL_CODE      0.000000
MOTOR_GAS_TYPE    12.156758
GEAR_BOX_TYPE     15.033382
dtype: float64

In [ ]:
df_vehicle['TRAFFIC_DATE'] = pd.to_datetime(df_vehicle['TRAFFIC_DATE'])
df_vehicle['TRAFFIC_YEAR'] = df_vehicle['TRAFFIC_DATE'].dt.year

# df_vehicle['TRAFFIC_YEAR'].fillna(df_vehicle['TRAFFIC_YEAR'].median(), inplace=True)
# df_vehicle.isna().mean() * 100

In [ ]:
df_vehicle.head(4)

,Unnamed: 0,Unnamed: 0.1,VEHICLE_ID,TRAFFIC_DATE,BRAND_CODE,BASEMODEL_CODE,TOPMODEL_CODE,MOTOR_GAS_TYPE,GEAR_BOX_TYPE,TRAFFIC_YEAR
0,0,5317713,5317713,2017-12-07,Z,13398,9349,Benzin,MANUEL,2017
1,1,5259212,5259212,2014-12-09,X,15694,9524,Dizel,OTOMATİK,2014
2,2,5110414,5110414,2005-12-12,Y,11183,9118,Benzin,OTOMATİK,2005
3,3,5101263,5101263,2005-08-23,X,15039,9495,Benzin,MANUEL,2005


### Convert Traffic_Year to Age

In [ ]:
# for index, row in df_vehicle.iterrows():
#   year_value = row['TRAFFIC_YEAR']
#   df_vehicle['AGE'].at[index] = current_year - year_value #current_year = 2021


df_vehicle['AGE'] = df_vehicle['TRAFFIC_YEAR'].map(lambda year: (current_year-year) )

In [ ]:
current_year

2021

In [ ]:
df_vehicle.head(10)

,Unnamed: 0,Unnamed: 0.1,VEHICLE_ID,TRAFFIC_DATE,BRAND_CODE,BASEMODEL_CODE,TOPMODEL_CODE,MOTOR_GAS_TYPE,GEAR_BOX_TYPE,TRAFFIC_YEAR,AGE
0,0,5317713,5317713,2017-12-07,Z,13398,9349,Benzin,MANUEL,2017,0
1,1,5259212,5259212,2014-12-09,X,15694,9524,Dizel,OTOMATİK,2014,0
2,2,5110414,5110414,2005-12-12,Y,11183,9118,Benzin,OTOMATİK,2005,0
3,3,5101263,5101263,2005-08-23,X,15039,9495,Benzin,MANUEL,2005,0
4,4,5053909,5053909,2003-04-09,Y,12255,9219,Dizel,OTOMATİK,2003,0
5,5,5002457,5002457,2000-12-31,X,14927,9491,NaN,NaN,2000,0
6,6,5069112,5069112,2004-07-20,Y,11201,9118,Benzin,MANUEL,2004,0
7,7,5149542,5149542,2008-01-07,T,13927,9384,Dizel,MANUEL,2008,0
8,8,5344597,5344597,2021-03-16,Z,12652,9245,Benzin,MANUEL,2021,0
9,9,5117495,5117495,2006-03-28,Y,11251,9119,Benzin,MANUEL,2006,0


In [ ]:
F_VEHICLE_DATA = F_VEHICLE_DATA.drop('TRAFFIC_DATE', axis=1)

### Deal with Unique Values

In [ ]:
df_vehicle['GEAR_BOX_TYPE'].unique()

array(['Manuel', 'Otomatik (DSG)', 'Otomatik', nan, 'Mekanik',
       'Sürekli Değişken', 'Otomaik', '?automatisch', 'Otomatik Değişken',
       'Sürekli değişken', 'Otomatik (DSG Şanzuman)', 'Düz',
       'Otomatik(DSG)', '?Handschaltung', 'manuel'], dtype=object)

# CUST_RELATED_DATA Processing

In [ ]:
df_cust_rel = F_CUST_RELATED_DATA.drop('FK_RELATION_STATUS_EXPLANATION', axis=1)

df_cust_rel['START_DATE'] = pd.to_datetime(df_cust_rel['START_DATE'])
df_cust_rel['END_DATE'] = pd.to_datetime(df_cust_rel['END_DATE'])

df_cust_rel['START_YEAR'] = df_cust_rel['START_DATE'].dt.year
df_cust_rel['END_YEAR'] = df_cust_rel['END_DATE'].dt.year

df_cust_rel = df_cust_rel.drop('END_DATE', axis=1)

df_cust_rel.head(3)

,Unnamed: 0,BASE_CUSTOMER_ID,VEHICLE_ID,START_DATE,FK_RELATION_STATUS_ID,START_YEAR,END_YEAR
0,0,189537,5321560,2018-07-24,2,2018,2019.0
1,1,110802,5329278,2019-07-16,1,2019,NaN
2,2,160615,5329282,2019-09-12,1,2019,NaN


In [ ]:
for index, row in df_cust_rel.iterrows():
  if row.FK_RELATION_STATUS_ID == 2:
    df_cust_rel.at[index, 'TOTAL_LICENCE_TIME'] = row.END_YEAR - row.START_YEAR
  if row.FK_RELATION_STATUS_ID == 1:
    df_cust_rel.at[index,'TOTAL_LICENCE_TIME'] = current_year - row.START_YEAR 

In [ ]:
df_cust_rel.head(2)

,Unnamed: 0,BASE_CUSTOMER_ID,VEHICLE_ID,START_DATE,FK_RELATION_STATUS_ID,START_YEAR,END_YEAR,TOTAL_LICENCE_TIME
0,0,189537,5321560,2018-07-24,2,2018,2019.0,1.0
1,1,110802,5329278,2019-07-16,1,2019,NaN,2.0


In [ ]:
df_cust_rel = df_cust_rel.drop(['START_YEAR', 'END_YEAR'], axis=1)
df_cust_rel.head(2)

,Unnamed: 0,BASE_CUSTOMER_ID,VEHICLE_ID,START_DATE,FK_RELATION_STATUS_ID,TOTAL_LICENCE_TIME
0,0,189537,5321560,2018-07-24,2,1.0
1,1,110802,5329278,2019-07-16,1,2.0


In [ ]:
df_cust_rel = df_cust_rel.drop('START_DATE', axis=1)

In [ ]:
df_cust_rel.drop( df_cust_rel.columns[0], axis=1).to_csv('drive/MyDrive/df_cust_rel.csv')

# SALES_DATA Processing

In [ ]:
df_sales = F_SALES_DATA
df_sales.head(2)

,Unnamed: 0,CUSTOMER_ID,SALESFILE_ID,SF_CREATE_DATE,STATUS,REQ_BRAND_CODE,REQ_TOPMODEL_CODE,CREATE_YEAR,CREATE_MONTH
0,0,1712792,7274467.0,2017-11-17 12:15:39,101,Y,9231.0,2017,11
1,1,1149729,4839430.0,2015-11-11 17:03:28,102,K,9184.0,2015,11


In [ ]:
df_sales.nunique()

Unnamed: 0           397526
CUSTOMER_ID          183523
SALESFILE_ID         396278
SF_CREATE_DATE       395534
STATUS                    6
REQ_BRAND_CODE            5
REQ_TOPMODEL_CODE       277
CREATE_YEAR              12
CREATE_MONTH             12
dtype: int64

## Sales Date Conversion

In [ ]:
df_sales['SF_CREATE_DATE'] = pd.to_datetime(df_sales['SF_CREATE_DATE'])
df_sales['CREATE_YEAR'] = df_sales['SF_CREATE_DATE'].dt.year
df_sales['CREATE_MONTH'] = df_sales['SF_CREATE_DATE'].dt.month
df_sales.head(2)

,Unnamed: 0,CUSTOMER_ID,SALESFILE_ID,SF_CREATE_DATE,STATUS,REQ_BRAND_CODE,REQ_TOPMODEL_CODE,CREATE_YEAR,CREATE_MONTH
0,0,1712792,7274467.0,2017-11-17 12:15:39,101,Y,9231.0,2017,11
1,1,1149729,4839430.0,2015-11-11 17:03:28,102,K,9184.0,2015,11


In [ ]:
df_sales['SF_CREATE_DATE'] = pd.to_datetime(df_sales['SF_CREATE_DATE']).dt.date

In [ ]:
df_sales.head(1)

,Unnamed: 0,CUSTOMER_ID,SALESFILE_ID,SF_CREATE_DATE,STATUS,REQ_BRAND_CODE,REQ_TOPMODEL_CODE,CREATE_YEAR,CREATE_MONTH
0,0,1712792,7274467.0,2017-11-17,101,Y,9231.0,2017,11


In [ ]:
df_sales.to_csv('drive/MyDrive/df_sales.csv', index_label= df_sales.columns[0] )

# FINAL PROCESSING

In [ ]:
df_vehicle = pd.read_csv('drive/MyDrive/df_vehicle_filled.csv', index_col=0)
df_cust_rel = pd.read_csv('drive/MyDrive/df_cust_rel.csv', index_col=0)

In [ ]:
df_customer = pd.read_csv('drive/MyDrive/df_customer_filled_v2.csv', index_col=0)

In [ ]:
df_sales = pd.read_csv('drive/MyDrive/df_sales.csv', index_col=0)

In [ ]:
df_customer.nunique()

BASE_CUSTOMER_ID    196564
CUSTOMER_ID         849919
GENDER                   2
MARITAL_STATUS           2
OCCUPATION              63
AGE                    107
dtype: int64

In [ ]:
df_vehicle.nunique()

Unnamed: 0.1      345092
VEHICLE_ID        345092
TRAFFIC_DATE        7428
BRAND_CODE             5
BASEMODEL_CODE      2477
TOPMODEL_CODE        346
MOTOR_GAS_TYPE         4
GEAR_BOX_TYPE          2
dtype: int64

In [ ]:
df_cust_rel.nunique()

BASE_CUSTOMER_ID         196538
VEHICLE_ID               345092
START_DATE                 8132
FK_RELATION_STATUS_ID         2
TOTAL_LICENCE_TIME           31
dtype: int64

In [ ]:
df_sales.nunique()

Unnamed: 0.1         397526
CUSTOMER_ID          183523
SALESFILE_ID         396278
SF_CREATE_DATE         4117
STATUS                    6
REQ_BRAND_CODE            5
REQ_TOPMODEL_CODE       277
CREATE_YEAR              12
CREATE_MONTH             12
dtype: int64

## Check For Null Values (as percentage)

In [ ]:
df_customer.isna().mean()*100

BASE_CUSTOMER_ID     0.000000
CUSTOMER_ID          0.000000
GENDER               0.000235
MARITAL_STATUS       9.419839
OCCUPATION          28.400589
AGE                  6.789000
dtype: float64

In [ ]:
df_cust_rel.isna().mean()*100

BASE_CUSTOMER_ID         0.0
VEHICLE_ID               0.0
START_DATE               0.0
FK_RELATION_STATUS_ID    0.0
TOTAL_LICENCE_TIME       0.0
dtype: float64

In [ ]:
df_vehicle.isna().mean()*100

Unnamed: 0.1       0.000000
VEHICLE_ID         0.000000
TRAFFIC_DATE       0.000000
BRAND_CODE         0.000000
BASEMODEL_CODE     0.000000
TOPMODEL_CODE      0.000000
MOTOR_GAS_TYPE    12.156758
GEAR_BOX_TYPE     15.033382
dtype: float64

In [ ]:
df_sales.isna().mean()*100

Unnamed: 0.1         0.0000
CUSTOMER_ID          0.0000
SALESFILE_ID         0.0000
SF_CREATE_DATE       0.0000
STATUS               0.0000
REQ_BRAND_CODE       0.0000
REQ_TOPMODEL_CODE    0.0727
CREATE_YEAR          0.0000
CREATE_MONTH         0.0000
dtype: float64

In [ ]:
F_SIFIR_ARAC_DATA.isna().mean()*100

Unnamed: 0     0.0
VEHICLE_ID     0.0
CUSTOMER_ID    0.0
CREATE_DATE    0.0
dtype: float64

## Fill the Null Values

### Fill CUSTOMER Data

Drop the OCCUPATION from CUSTOMER since 28% of it are null.

In [ ]:
df_customer = df_customer.drop('OCCUPATION', axis=1)

In [ ]:
df_customer.head(1)

,BASE_CUSTOMER_ID,CUSTOMER_ID,GENDER,MARITAL_STATUS,AGE
0,158891,1747700,Erkek,Evli,46.0


In [ ]:
df_customer = df_customer.drop('GENDER', axis=1)
df_customer.isna().mean()*100

BASE_CUSTOMER_ID    0.000000
CUSTOMER_ID         0.000000
MARITAL_STATUS      9.419839
AGE                 6.789000
dtype: float64

In [ ]:
cust_age_median = df_customer['AGE'].median()
cust_age_median

52.0

In [ ]:
df_customer['AGE'] = df_customer['AGE'].fillna( cust_age_median )
df_customer.isna().mean()*100

BASE_CUSTOMER_ID    0.000000
CUSTOMER_ID         0.000000
MARITAL_STATUS      9.419839
AGE                 0.000000
dtype: float64

In [ ]:
df_customer.dtypes

BASE_CUSTOMER_ID      int64
CUSTOMER_ID           int64
MARITAL_STATUS       object
AGE                 float64
dtype: object

In [ ]:
df_customer['AGE'] = df_customer.AGE.astype('uint16')

In [ ]:
df_customer.dtypes

BASE_CUSTOMER_ID     int64
CUSTOMER_ID          int64
MARITAL_STATUS      object
AGE                 uint16
dtype: object

In [ ]:
df_customer.head(1)

,BASE_CUSTOMER_ID,CUSTOMER_ID,MARITAL_STATUS,AGE
0,158891,1747700,Evli,46


In [ ]:
df_customer.isna().mean()*100

BASE_CUSTOMER_ID    0.000000
CUSTOMER_ID         0.000000
MARITAL_STATUS      9.419839
AGE                 0.000000
dtype: float64

While the median age of the 'Evli' people is 53, of the 'Bekar' ones is 42.

In [ ]:
evli_yas = df_customer[ df_customer['MARITAL_STATUS']=='Evli']['AGE'].median()
bekar_yas = df_customer[ df_customer['MARITAL_STATUS']=='Bekar']['AGE'].median()
evli_yas, bekar_yas

(53.0, 42.0)

In [ ]:
df_temp = df_customer.copy()

In [ ]:
evli = df_customer['AGE']>42
bekar = df_customer['AGE']<=42

df_customer.loc[evli, 'MARITAL_STATUS'] = df_customer.loc[evli, 'MARITAL_STATUS'].fillna('Evli')
df_customer.loc[bekar, 'MARITAL_STATUS'] = df_customer.loc[bekar, 'MARITAL_STATUS'].fillna('Bekar')

In [ ]:
df_customer.isna().mean()*100

BASE_CUSTOMER_ID    0.0
CUSTOMER_ID         0.0
MARITAL_STATUS      0.0
AGE                 0.0
dtype: float64

In [ ]:
df_customer.head(1)

,BASE_CUSTOMER_ID,CUSTOMER_ID,MARITAL_STATUS,AGE
0,158891,1747700,Evli,46


In [ ]:
F_CUST_DATA.shape[0], df_customer.shape[0]

(849919, 849919)

**NONE** of the data are omitted from CUSTOMER table

### Fill Sales Data

In [ ]:
df_sales.dropna(subset=['REQ_TOPMODEL_CODE'], inplace=True)
df_sales.isna().mean()*100

Unnamed: 0.1         0.0
CUSTOMER_ID          0.0
SALESFILE_ID         0.0
SF_CREATE_DATE       0.0
STATUS               0.0
REQ_BRAND_CODE       0.0
REQ_TOPMODEL_CODE    0.0
CREATE_YEAR          0.0
CREATE_MONTH         0.0
dtype: float64

## One Hot Encoding

### One Hot CUSTOMER

In [ ]:
# # importing OneHotEncoder
# from sklearn.preprocessing import OneHotEncoder()

In [ ]:
df_customer.head(1)

,BASE_CUSTOMER_ID,CUSTOMER_ID,MARITAL_STATUS,AGE
0,158891,1747700,Evli,46


In [ ]:
df_customer_dum = pd.get_dummies(df_customer, columns=['MARITAL_STATUS'])
df_customer_dum.head(5)

,BASE_CUSTOMER_ID,CUSTOMER_ID,AGE,MARITAL_STATUS_Bekar,MARITAL_STATUS_Evli
0,158891,1747700,46,0,1
1,169233,1710561,59,0,1
2,30887,1043923,50,0,1
3,38013,1101926,52,0,1
4,157091,1819787,72,0,1


### One Hot CUSTOMER_RELATED

## Save the Final DataFrames

In [ ]:
df_cust_rel.to_csv('drive/MyDrive/df_final_cust_rel.csv')
df_sales.to_csv('drive/MyDrive/df_final_sales.csv')
df_vehicle.to_csv('drive/MyDrive/df_final_vehicle.csv')

In [ ]:
df_customer_dum.to_csv('drive/MyDrive/df_final_customer.csv')

# MERGING PREPROCESSED DATA TO FEED THE MODEL

In [ ]:
df_cust_rel = pd.read_csv('drive/MyDrive/df_final_cust_rel.csv')
df_customer = pd.read_csv('drive/MyDrive/df_final_customer.csv')
df_sales = pd.read_csv('drive/MyDrive/df_final_sales.csv')
df_vehicle = pd.read_csv('drive/MyDrive/df_final_vehicle.csv')

In [ ]:
# df_cust_and_cust_rel = df_customer_dum.merge(df_cust_rel, on='BASE_CUSTOMER_ID')
# df_cust_and_cust_rel.head(15)

In [ ]:
df_sales['SF_CREATE_DATE'] = pd.to_datetime( df_sales['SF_CREATE_DATE'] )

In [ ]:
max_year=2021
max_month=2

df_sales_maxed = df_sales[ (df_sales['CREATE_YEAR']<= max_year) &
                            (df_sales['CREATE_MONTH']<= max_month) ]

df_sales_maxed.agg([min,max])

,Unnamed: 0,CUSTOMER_ID,SALESFILE_ID,SF_CREATE_DATE,STATUS,REQ_BRAND_CODE,REQ_TOPMODEL_CODE,CREATE_YEAR,CREATE_MONTH
min,28,1006448,100011.0,2010-01-21,100,K,9083.0,2010,1
max,397518,1841775,10062939.0,2021-02-28,109,Z,9974.0,2021,2


In [ ]:
max_date_dict = {'year': [max_year], 'month': [max_month], 'day':[1]}
df_max_date = pd.DataFrame(max_date_dict)
max_date = pd.to_datetime(df_max_date)

print('max_date=',max_date[0])

df_sales_y = df_sales[ ( df_sales['SF_CREATE_DATE']> max_date[0] ) &
                            (df_sales['SF_CREATE_DATE']< max_date[0] + pd.DateOffset(months=3))]

df_sales_y.agg([min,max])

max_date= 2021-02-01 00:00:00


,Unnamed: 0,CUSTOMER_ID,SALESFILE_ID,SF_CREATE_DATE,STATUS,REQ_BRAND_CODE,REQ_TOPMODEL_CODE,CREATE_YEAR,CREATE_MONTH
min,34,1026827,9961634.0,2021-02-02,100,K,9231.0,2021,2
max,397385,1843758,10320545.0,2021-04-30,109,Z,9970.0,2021,4


## Adding Sales File Info to DF_FINAL

In [ ]:
import time
start_time = time.time()

df_final = df_customer.copy() # We will go based on df_customer

# Create new columns by setting their value to 0
df_final['TOTAL_SALES_FILE_3M'] = 0 # Sale Values of Last 3 Months 
df_final['SINCE_LAST_SALES_FILE_3M'] = 0 # The last sale record since 3 months

df_final['TOTAL_SALES_FILE_1Y'] = 0 # Last 1 Year
df_final['SINCE_LAST_SALES_FILE_1Y'] = 0

df_final['TOTAL_SALES_FILE_ALL'] = 0 # All Times
df_final['SINCE_LAST_SALES_FILE_ALL'] = 0

for index, row in df_customer.iterrows():
  df_sls_cust = df_sales_maxed[ df_sales_maxed['CUSTOMER_ID']
                                        ==row.loc['CUSTOMER_ID'] ]

  c_date = df_sls_cust['SF_CREATE_DATE'] # Create Date
  df_sls_cust_3m = df_sls_cust[ c_date > max_date[0] - pd.DateOffset(months=3) ]
  df_sls_cust_1y = df_sls_cust[ c_date > max_date[0] - pd.DateOffset(years=1) ]

  sale_count_3m = df_sls_cust_3m.shape[0]
  sale_count_1y = df_sls_cust_1y.shape[0]
  sale_count_all = df_sls_cust.shape[0] # All Sale Values Until Max Date

  if sale_count_3m > 0:
    # Take the sale that has the biggest datetime value (last)
    last_sale_3m = df_sls_cust_3m.sort_values(by='SF_CREATE_DATE', 
                                        ascending=False).iloc[0]['SF_CREATE_DATE']

    df_final.at[index, 'TOTAL_SALES_FILE_3M'] = sale_count_3m

    date_difference_3m = (max_date - last_sale_3m)[0].days/30
    date_difference_3m = int(date_difference_3m)

    df_final.at[index, 'SINCE_LAST_SALES_FILE_3M'] = date_difference_3m

  if sale_count_1y > 0:
    last_sale_1y = df_sls_cust_1y.sort_values(by='SF_CREATE_DATE', 
                                  ascending=False).iloc[0]['SF_CREATE_DATE']
    df_final.at[index, 'TOTAL_SALES_FILE_1Y'] = sale_count_1y

    date_difference_1y = (max_date - last_sale_1y)[0].days/30
    date_difference_1y = int(date_difference_1y)

    df_final.at[index, 'SINCE_LAST_SALES_FILE_1Y'] = date_difference_1y

  if sale_count_all > 0:
    last_sale_all = df_sls_cust.sort_values(by='SF_CREATE_DATE', 
                                  ascending=False).iloc[0]['SF_CREATE_DATE']
    df_final.at[index, 'TOTAL_SALES_FILE_ALL'] = sale_count_all
    
    date_difference_all = (max_date - last_sale_all)[0].days/30
    date_difference_all = int(date_difference_all)

    df_final.at[index, 'SINCE_LAST_SALES_FILE_ALL'] = date_difference_all
  
  ## y values
  df_sales_temp = df_sales_y[ df_sales_y['CUSTOMER_ID']
                                        ==row.loc['CUSTOMER_ID'] ]
  sales_count_y = df_sales_temp.shape[0]

  if sales_count_y > 0:
    df_final.at[index, 'HAS_SALES_IN_INTERVAL'] = 1
  


try:
  print("--- The code executed in %s minutes ---" % (time.time() - start_time)/60)
except:
  pass

    # display(row)
    # display(df_sls_cust_3m)
    # display(df_final.iloc[index])
    # count += 1
    # break

# print('count=',count)

In [ ]:
df_final.isna().sum()

Unnamed: 0                        0
BASE_CUSTOMER_ID                  0
CUSTOMER_ID                       0
AGE                               0
MARITAL_STATUS_Bekar              0
MARITAL_STATUS_Evli               0
TOTAL_SALES_FILE_3M               0
SINCE_LAST_SALES_FILE_3M          0
TOTAL_SALES_FILE_1Y               0
SINCE_LAST_SALES_FILE_1Y          0
TOTAL_SALES_FILE_ALL              0
SINCE_LAST_SALES_FILE_ALL         0
HAS_SALES_IN_INTERVAL        842300
dtype: int64

In [ ]:
# df_final.to_csv('drive/MyDrive/DF_FINAL.csv')

## Creating y Labels

In [ ]:
df_final = pd.read_csv('drive/MyDrive/DF_FINAL.csv')

In [ ]:
df_final.shape[0]

849919

In [ ]:
df_final.isna().sum()

Unnamed: 0                        0
Unnamed: 0.1                      0
BASE_CUSTOMER_ID                  0
CUSTOMER_ID                       0
AGE                               0
MARITAL_STATUS_Bekar              0
MARITAL_STATUS_Evli               0
TOTAL_SALES_FILE_3M               0
SINCE_LAST_SALES_FILE_3M          0
TOTAL_SALES_FILE_1Y               0
SINCE_LAST_SALES_FILE_1Y          0
TOTAL_SALES_FILE_ALL              0
SINCE_LAST_SALES_FILE_ALL         0
HAS_SALES_IN_INTERVAL        842300
dtype: int64

In [ ]:
df_final.sample(1)

,Unnamed: 0,Unnamed: 0.1,BASE_CUSTOMER_ID,CUSTOMER_ID,AGE,MARITAL_STATUS_Bekar,MARITAL_STATUS_Evli,TOTAL_SALES_FILE_3M,SINCE_LAST_SALES_FILE_3M,TOTAL_SALES_FILE_1Y,SINCE_LAST_SALES_FILE_1Y,TOTAL_SALES_FILE_ALL,SINCE_LAST_SALES_FILE_ALL,HAS_SALES_IN_INTERVAL
763990,763990,763990,170406,1519052,40,0,1,0,0,0,0,0,0,NaN


In [ ]:
df_final[ df_final['BASE_CUSTOMER_ID']==36558]

,Unnamed: 0,Unnamed: 0.1,BASE_CUSTOMER_ID,CUSTOMER_ID,AGE,MARITAL_STATUS_Bekar,MARITAL_STATUS_Evli,TOTAL_SALES_FILE_3M,SINCE_LAST_SALES_FILE_3M,TOTAL_SALES_FILE_1Y,SINCE_LAST_SALES_FILE_1Y,TOTAL_SALES_FILE_ALL,SINCE_LAST_SALES_FILE_ALL,HAS_SALES_IN_INTERVAL
221951,221951,221951,36558,1645359,53,0,1,0,0,0,0,0,0,NaN
297212,297212,297212,36558,1514328,53,0,1,0,0,0,0,2,12,NaN
368037,368037,368037,36558,1412107,53,0,1,0,0,0,0,0,0,NaN
544500,544500,544500,36558,1813090,53,0,1,0,0,0,0,0,0,NaN
548784,548784,548784,36558,1210108,53,0,1,0,0,0,0,0,0,NaN
633844,633844,633844,36558,1156978,53,0,1,0,0,0,0,0,0,NaN
696121,696121,696121,36558,1599599,53,0,1,0,0,0,0,1,96,NaN


In [ ]:
not_has_sales = df_final['HAS_SALES_IN_INTERVAL']!=1

df_final.loc[
  not_has_sales, 'HAS_SALES_IN_INTERVAL'] = df_final.loc[ 
                                              not_has_sales, 'HAS_SALES_IN_INTERVAL'].fillna('0')


In [ ]:
df_final[ df_final['HAS_SALES_IN_INTERVAL']==1].head(5)

,Unnamed: 0,Unnamed: 0.1,BASE_CUSTOMER_ID,CUSTOMER_ID,AGE,MARITAL_STATUS_Bekar,MARITAL_STATUS_Evli,TOTAL_SALES_FILE_3M,SINCE_LAST_SALES_FILE_3M,TOTAL_SALES_FILE_1Y,SINCE_LAST_SALES_FILE_1Y,TOTAL_SALES_FILE_ALL,SINCE_LAST_SALES_FILE_ALL,HAS_SALES_IN_INTERVAL
15,15,15,84171,1841096,56,0,1,0,0,0,0,0,0,1
107,107,107,137731,1391196,38,0,1,2,0,2,0,2,0,1
279,279,279,66672,1843263,71,0,1,0,0,0,0,0,0,1
373,373,373,74962,1842664,45,1,0,0,0,0,0,0,0,1
472,472,472,171447,1522723,39,0,1,0,0,0,0,0,0,1


# PREPARE THE TEST DATASET WITH THE DESIRED INTERVAL

In [ ]:
# df_final_merge = pd.read_csv('drive/MyDrive/DF_FINAL_MERGED_v2.csv')

In [ ]:
df_cust_rel = pd.read_csv('drive/MyDrive/df_final_cust_rel.csv')
df_customer = pd.read_csv('drive/MyDrive/df_final_customer.csv')
df_sales = pd.read_csv('drive/MyDrive/df_final_sales.csv')
df_vehicle = pd.read_csv('drive/MyDrive/df_final_vehicle.csv')

In [ ]:
df_service = pd.read_csv('drive/MyDrive/slm/CUSTOMER_RELATED_MONEY.csv')

In [ ]:
df_sales['SF_CREATE_DATE'] = pd.to_datetime( df_sales['SF_CREATE_DATE'] )

In [ ]:
max_year=2021
max_month=3 # MAX TRAIN MONTH

df_sales_maxed = df_sales[ (df_sales['CREATE_YEAR']<= max_year) &
                            (df_sales['CREATE_MONTH']<= max_month) ]

df_sales_maxed.agg([min,max])

,Unnamed: 0,CUSTOMER_ID,SALESFILE_ID,SF_CREATE_DATE,STATUS,REQ_BRAND_CODE,REQ_TOPMODEL_CODE,CREATE_YEAR,CREATE_MONTH
min,28,1003463,100011.0,2010-01-21,100,K,9083.0,2010,1
max,397518,1842740,10223380.0,2021-03-31,109,Z,9974.0,2021,3


In [ ]:
max_date_dict = {'year': [max_year], 'month': [max_month], 'day':[1]}
df_max_date = pd.DataFrame(max_date_dict)
max_date = pd.to_datetime(df_max_date)

print('max_date=',max_date[0])

max_date= 2021-03-01 00:00:00


## Adding SIFIR_ARAC as Feature

In [ ]:
df_sifir = F_SIFIR_ARAC_DATA
df_sifir.head(2)

,Unnamed: 0,VEHICLE_ID,CUSTOMER_ID,CREATE_DATE
0,0,5015193,1063740,2005-01-01 19:08:33
1,1,5110231,1155678,2005-09-29 17:16:15


In [ ]:
df_sifir['CREATE_DATE'] = pd.to_datetime( df_sifir['CREATE_DATE'] )

In [ ]:
df_sifir_maxed = df_sifir[ df_sifir['CREATE_DATE'] < max_date[0]]

df_sifir_maxed.agg([min,max])

,Unnamed: 0,VEHICLE_ID,CUSTOMER_ID,CREATE_DATE
min,0,5000015,1000520,2005-01-01 14:46:45
max,169524,5344593,1840887,2021-02-26 21:03:40


## Adding y Values

In [ ]:
df_sales_y = df_sales[ ( df_sales['SF_CREATE_DATE']> max_date[0] ) &
                            (df_sales['SF_CREATE_DATE']< max_date[0] + pd.DateOffset(months=3))]

df_sales_y.agg([min,max])

,Unnamed: 0,CUSTOMER_ID,SALESFILE_ID,SF_CREATE_DATE,STATUS,REQ_BRAND_CODE,REQ_TOPMODEL_CODE,CREATE_YEAR,CREATE_MONTH
min,50,1026751,10069150.0,2021-03-02,100,K,9233.0,2021,3
max,397385,1845569,10391897.0,2021-05-31,109,Z,9970.0,2021,5


## Adding Service

In [ ]:
df_service.head(2)

,Unnamed: 0,BASE_CUSTOMER_ID,VEHICLE_ID,START_DATE,END_DATE,FK_RELATION_STATUS_ID,FK_RELATION_STATUS_EXPLANATION,PAID_MONEY
0,0,189537,5321560,2018-07-24 00:00:00.000000,2019-07-03 00:00:00.000000,2,Pasif Ruhsat Sahibi,NaN
1,1,110802,5329278,2019-07-16 00:00:00.000000,2021-10-31 13:14:23.956935,1,Aktif Ruhsat Sahibi,21522.0


In [ ]:
df_service['START_DATE'] = pd.to_datetime( df_service['START_DATE'] )
df_service['END_DATE'] = pd.to_datetime( df_service['END_DATE'] )

## The main feature addition function

In [ ]:
import time
from IPython.display import clear_output

start_time = time.time()

df_final = df_customer.copy() # We will go based on df_customer

# Create new columns by setting their value to 0
df_final['TOTAL_SALES_FILE_3M'] = 0 # Sale Values of Last 3 Months 
df_final['SINCE_LAST_SALES_FILE_3M'] = 0 # The last sale record since 3 months

df_final['TOTAL_SALES_FILE_1Y'] = 0 # Last 1 Year
df_final['SINCE_LAST_SALES_FILE_1Y'] = 0

df_final['TOTAL_SALES_FILE_ALL'] = 0 # All Times
df_final['SINCE_LAST_SALES_FILE_ALL'] = 0

#SIFIR ARAC
df_final['TOTAL_SIFIR_ARAC_3M'] = 0
df_final['SINCE_LAST_SIFIR_ARAC_3M'] = 0

df_final['TOTAL_SIFIR_ARAC_1Y'] = 0
df_final['SINCE_LAST_SIFIR_ARAC_1Y'] = 0

df_final['TOTAL_SIFIR_ARAC_ALL'] = 0
df_final['SINCE_LAST_SIFIR_ARAC_ALL'] = 0

#SERVICE
# df_final['TOTAL_PAID_MONEY'] = 0
# df_final['MONEY_PER_MONTH']
df_final['']


# y values
df_final['HAS_SALES_IN_INTERVAL'] = 0


count=0
for index, row in df_customer.iterrows():
  df_sls_cust = df_sales_maxed[ df_sales_maxed['CUSTOMER_ID']
                                        ==row.loc['CUSTOMER_ID'] ]

  c_date = df_sls_cust['SF_CREATE_DATE'] # Create Date
  df_sls_cust_3m = df_sls_cust[ c_date > max_date[0] - pd.DateOffset(months=3) ]
  df_sls_cust_1y = df_sls_cust[ c_date > max_date[0] - pd.DateOffset(years=1) ]

  sale_count_3m = df_sls_cust_3m.shape[0]
  sale_count_1y = df_sls_cust_1y.shape[0]
  sale_count_all = df_sls_cust.shape[0] # All Sale Values Until Max Date

  if sale_count_3m > 0:
    # Take the sale that has the biggest datetime value (last)
    last_sale_3m = df_sls_cust_3m.sort_values(by='SF_CREATE_DATE', 
                                        ascending=False).iloc[0]['SF_CREATE_DATE']

    df_final.at[index, 'TOTAL_SALES_FILE_3M'] = sale_count_3m

    date_difference_3m = (max_date - last_sale_3m)[0].days/30
    date_difference_3m = int(date_difference_3m)

    df_final.at[index, 'SINCE_LAST_SALES_FILE_3M'] = date_difference_3m

  if sale_count_1y > 0:
    last_sale_1y = df_sls_cust_1y.sort_values(by='SF_CREATE_DATE', 
                                  ascending=False).iloc[0]['SF_CREATE_DATE']
    df_final.at[index, 'TOTAL_SALES_FILE_1Y'] = sale_count_1y

    date_difference_1y = (max_date - last_sale_1y)[0].days/30
    date_difference_1y = int(date_difference_1y)

    df_final.at[index, 'SINCE_LAST_SALES_FILE_1Y'] = date_difference_1y

  if sale_count_all > 0:
    last_sale_all = df_sls_cust.sort_values(by='SF_CREATE_DATE', 
                                  ascending=False).iloc[0]['SF_CREATE_DATE']
    df_final.at[index, 'TOTAL_SALES_FILE_ALL'] = sale_count_all
    
    date_difference_all = (max_date - last_sale_all)[0].days/30
    date_difference_all = int(date_difference_all)

    df_final.at[index, 'SINCE_LAST_SALES_FILE_ALL'] = date_difference_all

    df_sls_cust = df_sales_maxed[ df_sales_maxed['CUSTOMER_ID']
                                        ==row.loc['CUSTOMER_ID'] ]


  # SIFIR_ARAC
  df_sifir_cust = df_sifir_maxed[ df_sifir_maxed['CUSTOMER_ID']
                                        ==row.loc['CUSTOMER_ID'] ]

  c_date = df_sifir_cust['CREATE_DATE'] # Create Date
  df_sifir_cust_3m = df_sifir_cust[ c_date > max_date[0] - pd.DateOffset(months=3) ]
  df_sifir_cust_1y = df_sifir_cust[ c_date > max_date[0] - pd.DateOffset(years=1) ]

  sifir_count_3m = df_sifir_cust_3m.shape[0]
  sifir_count_1y = df_sifir_cust_1y.shape[0]
  sifir_count_all = df_sifir_cust.shape[0] # All Sale Values Until Max Date

  if sifir_count_3m > 0:
    # Take the sifir_arac that has the biggest datetime value (last)
    last_sifir_3m = df_sifir_cust_3m.sort_values(by='CREATE_DATE', 
                                        ascending=False).iloc[0]['CREATE_DATE']

    df_final.at[index, 'TOTAL_SIFIR_ARAC_3M'] = sifir_count_3m

    date_difference_3m = (max_date - last_sifir_3m)[0].days/30
    date_difference_3m = int(date_difference_3m)

    df_final.at[index, 'SINCE_LAST_SIFIR_ARAC_3M'] = date_difference_3m


  if sifir_count_1y > 0:
    # Take the sifir_arac that has the biggest datetime value (last)
    last_sifir_1y = df_sifir_cust_1y.sort_values(by='CREATE_DATE', 
                                        ascending=False).iloc[0]['CREATE_DATE']

    df_final.at[index, 'TOTAL_SIFIR_ARAC_1Y'] = sifir_count_1y

    date_difference_1y = (max_date - last_sifir_1y)[0].days/30
    date_difference_1y = int(date_difference_1y)

    df_final.at[index, 'SINCE_LAST_SIFIR_ARAC_1Y'] = date_difference_1y

  if sifir_count_all > 0:
    # Take the sifir_arac that has the biggest datetime value (last)
    last_sifir_all = df_sifir_cust.sort_values(by='CREATE_DATE', 
                                        ascending=False).iloc[0]['CREATE_DATE']

    df_final.at[index, 'TOTAL_SIFIR_ARAC_ALL'] = sifir_count_all

    date_difference_all = (max_date - last_sifir_all)[0].days/30
    date_difference_all = int(date_difference_all)

    df_final.at[index, 'SINCE_LAST_SIFIR_ARAC_ALL'] = date_difference_all


    ## y values
  df_sales_temp = df_sales_y[ df_sales_y['CUSTOMER_ID']
                                        ==row.loc['CUSTOMER_ID'] ]
  sales_count_y = df_sales_temp.shape[0]

  if sales_count_y > 0:
    df_final.at[index, 'HAS_SALES_IN_INTERVAL'] = 1


  count += 1
  if count % 3000 ==0:
    clear_output(wait=False)
    percentage = (count/df_customer.shape[0])*100
    print('Completed=', percentage , '%')

  # display( df_final[ df_final['HAS_SALES_IN_INTERVAL']==1 ] )
  # if count > 150:
  #   break


Completed= 99.89187204898349 %


In [ ]:
df_final.to_csv('drive/MyDrive/DF_FINAL_RAW_wSIFIR_2021_3_y.csv')

In [ ]:
exec_time = (time.time() - start_time)/60
print("--- The code executed in %s minutes ---" % exec_time )

--- The code executed in 82.69094372987747 minutes ---


# Merge DF_FINAL Based on BASE_CUSTOMER_ID

In [ ]:
df_final = pd.read_csv('drive/MyDrive/DF_FINAL_RAW_wSIFIR_2021_3_y.csv')

In [ ]:

import time
start_time = time.time()

emp_list = np.zeros( (1,df_final.shape[1]-2), dtype='uint32') # Skip Unnamed Columns

my_df_list = []
my_bas_cust_id_list = []

#### FIRST ROW WILL BE OMITTED ####

#### CUSTOMER_ID WILL BE OMITTED ####

# df_final_merged = pd.DataFrame(emp_list, columns= df_final.columns[2:])
count=0

pd.options.mode.chained_assignment = None  # default='warn'

for index, row in df_final.iterrows():
  df_bc = df_final[ df_final['BASE_CUSTOMER_ID']
                                        ==row.loc['BASE_CUSTOMER_ID'] ]
  base_id = df_bc.iloc[0]['BASE_CUSTOMER_ID']

  if base_id not in my_bas_cust_id_list:
    my_bas_cust_id_list.append(base_id)
    df_bc['HAS_SALES_IN_INTERVAL'] = pd.to_numeric(df_bc['HAS_SALES_IN_INTERVAL'])
    
    if df_bc.shape[0] > 0:
      df_temp = pd.DataFrame(emp_list, columns= df_final.columns[2:] )

      df_temp.at[0,'BASE_CUSTOMER_ID'] = base_id
      df_temp.at[0,'AGE'] = df_bc.iloc[0]['AGE']
      df_temp.at[0,'MARITAL_STATUS_Bekar'] =df_bc.iloc[0]['MARITAL_STATUS_Bekar']
      df_temp.at[0, 'MARITAL_STATUS_Evli'] = df_bc.iloc[0]['MARITAL_STATUS_Evli']

      df_temp.at[0, 'TOTAL_SALES_FILE_3M'] = df_bc['TOTAL_SALES_FILE_3M'].sum()
      df_temp.at[0, 'SINCE_LAST_SALES_FILE_3M'] = df_bc['SINCE_LAST_SALES_FILE_3M'].min()
      df_temp.at[0, 'TOTAL_SALES_FILE_1Y'] = df_bc['TOTAL_SALES_FILE_1Y'].sum()
      df_temp.at[0, 'SINCE_LAST_SALES_FILE_1Y'] = df_bc['SINCE_LAST_SALES_FILE_1Y'].min()
      df_temp.at[0, 'TOTAL_SALES_FILE_ALL'] = df_bc['TOTAL_SALES_FILE_ALL'].sum()
      df_temp.at[0, 'SINCE_LAST_SALES_FILE_ALL'] = df_bc['SINCE_LAST_SALES_FILE_ALL'].min()

      df_temp.at[0, 'TOTAL_SIFIR_ARAC_3M'] = df_bc['TOTAL_SIFIR_ARAC_3M'].sum()
      df_temp.at[0, 'SINCE_LAST_SIFIR_ARAC_3M'] = df_bc['SINCE_LAST_SIFIR_ARAC_3M'].min()
      df_temp.at[0, 'TOTAL_SIFIR_ARAC_1Y'] = df_bc['TOTAL_SIFIR_ARAC_1Y'].sum()
      df_temp.at[0, 'SINCE_LAST_SIFIR_ARAC_1Y'] = df_bc['SINCE_LAST_SIFIR_ARAC_1Y'].min()
      df_temp.at[0, 'TOTAL_SIFIR_ARAC_ALL'] = df_bc['TOTAL_SIFIR_ARAC_ALL'].sum()
      df_temp.at[0, 'SINCE_LAST_SIFIR_ARAC_ALL'] = df_bc['SINCE_LAST_SIFIR_ARAC_ALL'].min()

      df_temp.at[0,'HAS_SALES_IN_INTERVAL'] = df_bc['HAS_SALES_IN_INTERVAL'].max()
      my_df_list.append(df_temp.copy())

    count += 1

    if count % 3000 ==0:
      clear_output(wait=False)
      percentage = (count/df_final.shape[0])*100
      print('Completed=', percentage , '%')
      
end_time = time.time()

Completed= 22.94336283810575 %


In [ ]:
print("EXECUTION TIME=", (end_time-start_time)/60, ' MINUTES')

EXECUTION TIME= 114.02274867693583  MINUTES


In [ ]:
df_final_merge = pd.concat(my_df_list)

In [ ]:
df_final_merge.shape[0]

196564

In [ ]:
df_final_merge.head(5)

,BASE_CUSTOMER_ID,CUSTOMER_ID,AGE,MARITAL_STATUS_Bekar,MARITAL_STATUS_Evli,TOTAL_SALES_FILE_3M,SINCE_LAST_SALES_FILE_3M,TOTAL_SALES_FILE_1Y,SINCE_LAST_SALES_FILE_1Y,TOTAL_SALES_FILE_ALL,SINCE_LAST_SALES_FILE_ALL,TOTAL_SIFIR_ARAC_3M,SINCE_LAST_SIFIR_ARAC_3M,TOTAL_SIFIR_ARAC_1Y,SINCE_LAST_SIFIR_ARAC_1Y,TOTAL_SIFIR_ARAC_ALL,SINCE_LAST_SIFIR_ARAC_ALL,HAS_SALES_IN_INTERVAL
0,158891,0,46,0,1,0,0,0,0,0,0,0,0,1,0,2,0,0
0,169233,0,59,0,1,1,0,1,0,1,0,0,0,0,0,1,0,0
0,30887,0,50,0,1,0,0,0,0,4,0,0,0,0,0,9,0,0
0,38013,0,52,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
0,157091,0,72,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0


In [ ]:
df_final_merge.nunique()

BASE_CUSTOMER_ID             196564
CUSTOMER_ID                       1
AGE                             105
MARITAL_STATUS_Bekar              2
MARITAL_STATUS_Evli               2
TOTAL_SALES_FILE_3M              19
SINCE_LAST_SALES_FILE_3M          2
TOTAL_SALES_FILE_1Y              20
SINCE_LAST_SALES_FILE_1Y          2
TOTAL_SALES_FILE_ALL             49
SINCE_LAST_SALES_FILE_ALL         3
TOTAL_SIFIR_ARAC_3M               4
SINCE_LAST_SIFIR_ARAC_3M          1
TOTAL_SIFIR_ARAC_1Y              11
SINCE_LAST_SIFIR_ARAC_1Y          1
TOTAL_SIFIR_ARAC_ALL             46
SINCE_LAST_SIFIR_ARAC_ALL        21
HAS_SALES_IN_INTERVAL             2
dtype: int64

In [ ]:
df_final_merge.to_csv('drive/MyDrive/DF_FINAL_MERGED_wSIFIR_2021_3.csv')

In [ ]:
print('total iterated row=',count)

total iterated row= 196564


## Unnecessary


In [ ]:
max_year = 2021
max_month = 2 # as we already defined before

max_date_dict = {'year': [max_year], 'month': [max_month], 'day':[1]}
df_max_date = pd.DataFrame(max_date_dict)
max_date = pd.to_datetime(df_max_date)

df_sales['SF_CREATE_DATE'] = pd.to_datetime( df_sales['SF_CREATE_DATE'])


,Unnamed: 0,CUSTOMER_ID,SALESFILE_ID,SF_CREATE_DATE,STATUS,REQ_BRAND_CODE,REQ_TOPMODEL_CODE,CREATE_YEAR,CREATE_MONTH
min,34,1026827,9961634.0,2021-02-02,100,K,9231.0,2021,2
max,397385,1843758,10320545.0,2021-04-30,109,Z,9970.0,2021,4


In [ ]:
df_final['HAS_SALES_IN_INTERVAL'] = 0

for index, row in df_customer.iterrows():
  df_sales_temp = df_sales_y[ df_sales_y['CUSTOMER_ID']
                                        ==row.loc['CUSTOMER_ID'] ]
  sales_count = df_sales_temp.shape[0]

  if sales_count > 0:
    df_final.at[index, 'HAS_SALES_IN_INTERVAL'] = 1

In [ ]:
df_final[ df_final['HAS_SALES_IN_INTERVAL']==1]

,Unnamed: 0,Unnamed: 0.1,BASE_CUSTOMER_ID,CUSTOMER_ID,AGE,GENDER_Erkek,GENDER_Kadın,MARITAL_STATUS_Bekar,MARITAL_STATUS_Evli,TOTAL_SALES_FILE_3M,SINCE_LAST_SALES_FILE_3M,TOTAL_SALES_FILE_1Y,SINCE_LAST_SALES_FILE_1Y,SINCE_LAST_SALES_FILE_ALL,TOTAL_SALES_FILE_ALL,HAS_SALES_IN_INTERVAL
15,15.0,17.0,76345.0,1590300.0,69.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0
107,107.0,118.0,141024.0,1799695.0,32.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0
279,279.0,313.0,142255.0,1405811.0,61.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0
373,373.0,418.0,119062.0,1331692.0,63.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0
472,472.0,524.0,18502.0,1056948.0,60.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
849695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
849706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
849721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
849832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [ ]:
df_final.isna().sum()

Unnamed: 0                      742
Unnamed: 0.1                    742
BASE_CUSTOMER_ID                742
CUSTOMER_ID                     742
AGE                             742
GENDER_Erkek                    742
GENDER_Kadın                    742
MARITAL_STATUS_Bekar            742
MARITAL_STATUS_Evli             742
TOTAL_SALES_FILE_3M             742
SINCE_LAST_SALES_FILE_3M        742
TOTAL_SALES_FILE_1Y             742
SINCE_LAST_SALES_FILE_1Y        742
SINCE_LAST_SALES_FILE_ALL       742
TOTAL_SALES_FILE_ALL         727304
HAS_SALES_IN_INTERVAL             0
dtype: int64

In [ ]:
df_final.head(5)

,Unnamed: 0,Unnamed: 0.1,BASE_CUSTOMER_ID,CUSTOMER_ID,AGE,GENDER_Erkek,GENDER_Kadın,MARITAL_STATUS_Bekar,MARITAL_STATUS_Evli,TOTAL_SALES_FILE_3M,SINCE_LAST_SALES_FILE_3M,TOTAL_SALES_FILE_1Y,SINCE_LAST_SALES_FILE_1Y,SINCE_LAST_SALES_FILE_ALL,TOTAL_SALES_FILE_ALL,HAS_SALES_IN_INTERVAL
0,0.0,0.0,158891.0,1747700.0,46.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
1,1.0,1.0,169233.0,1710561.0,59.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
2,2.0,2.0,30887.0,1043923.0,50.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
3,3.0,4.0,157091.0,1819787.0,72.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
4,4.0,5.0,6590.0,1826742.0,69.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0


In [ ]:
df_final.agg([min,max])

,Unnamed: 0,BASE_CUSTOMER_ID,CUSTOMER_ID,AGE,GENDER_Erkek,GENDER_Kadın,MARITAL_STATUS_Bekar,MARITAL_STATUS_Evli,TOTAL_SALES_FILE_3M,SINCE_LAST_SALES_FILE_3M,TOTAL_SALES_FILE_1Y,SINCE_LAST_SALES_FILE_1Y,SINCE_LAST_SALES_FILE_ALL,TOTAL_SALES_FILE_ALL
min,0,2,1000002,0,0,0,0,0,0,0,0,0,0,1.0
max,849918,196564,1849919,107,1,1,1,1,13,0,13,12,134,51.0


# Exploration for Merging

In [ ]:
df_sales.head(1)

,Unnamed: 0,CUSTOMER_ID,SALESFILE_ID,SF_CREATE_DATE,STATUS,REQ_BRAND_CODE,REQ_TOPMODEL_CODE,CREATE_YEAR,CREATE_MONTH
0,0,1712792,7274467.0,2017-11-17 12:15:39,101,Y,9231.0,2017,11


In [ ]:
df_customer = F_CUST_DATA
df_customer[ df_customer['CUSTOMER_ID'] == 1712792]

,Unnamed: 0,BASE_CUSTOMER_ID,CUSTOMER_ID,GENDER,GENDER_ID,MARITAL_STATUS,MARITAL_STATUS_ID,BIRTH_DATE,FK_ADDRESS_COMMUNICATION_CITY,OCCUPATION
374633,374633,31024,1712792,Erkek,100,Evli,101,1971.0,Düzce,Diğer


In [ ]:
df_cust_rel = F_CUST_RELATED_DATA
df_cust_rel[ df_cust_rel['BASE_CUSTOMER_ID'] == 31024]
                .sort_values(by=['START_DATE'])

,Unnamed: 0,BASE_CUSTOMER_ID,VEHICLE_ID,START_DATE,END_DATE,FK_RELATION_STATUS_ID,FK_RELATION_STATUS_EXPLANATION
102956,102956,31024,5067245,2004-08-18,NaN,1,Aktif Ruhsat Sahibi
281601,281601,31024,5280177,2015-10-09,NaN,1,Aktif Ruhsat Sahibi
297993,297993,31024,5315137,2017-12-12,2020-04-30,2,Pasif Ruhsat Sahibi


In [ ]:
df_vehicle = F_VEHICLE_DATA.copy()
df_vehicle[df_vehicle['VEHICLE_ID'] == 5341920]

,Unnamed: 0,VEHICLE_ID,TRAFFIC_DATE,BRAND_CODE,BASEMODEL_CODE,TOPMODEL_CODE,MOTOR_GAS_TYPE,GEAR_BOX_TYPE
127894,127894,5341920,2021-01-25,Y,16518,9625,?Benzin,?automatisch


In [ ]:
# F_SIFIR_ARAC_DATA.head(6)
df_sifir = F_SIFIR_ARAC_DATA.copy()
df_sifir[ df_sifir['VEHICLE_ID']== 5341920]

,Unnamed: 0,VEHICLE_ID,CUSTOMER_ID,CREATE_DATE
133792,133792,5341920,1573659,2021-01-22 12:26:56


In [ ]:
df_vehicle.sample(n=3) # Get 3 Random Values

,Unnamed: 0,VEHICLE_ID,TRAFFIC_DATE,BRAND_CODE,BASEMODEL_CODE,TOPMODEL_CODE,MOTOR_GAS_TYPE,GEAR_BOX_TYPE
127894,127894,5341920,2021-01-25,Y,16518,9625,?Benzin,?automatisch
47881,47881,5113901,2006-02-21,T,13907,9383,Kurşunsuz Benzin,Otomatik
164611,164611,5342339,2021-03-19,Y,16442,9605,Benzin,Otomatik (DSG)


In [ ]:
sifir_arac_data = pd.merge(df,F_VEHICLE_DATA, on='VEHICLE_ID')

# BUILDING THE MODEL

In [ ]:
sample_data = pd.read_csv('sample_submission.csv.zip')

In [ ]:
sample_data

,Id,Expected
0,168254,NaN
1,124747,NaN
2,135493,NaN
3,90543,NaN
4,40421,NaN
...,...,...
166938,53945,NaN
166939,98842,NaN
166940,55766,NaN
166941,22680,NaN


# BUILDING NEURAL NETWORK MODEL

In [ ]:
df = pd.read_csv('drive/MyDrive/DF_FINAL_MERGED_wSIFIR_2021_3.csv')

In [ ]:
df.drop(['Unnamed: 0', 'BASE_CUSTOMER_ID', 'CUSTOMER_ID'], axis=1, inplace=True)
y = df['HAS_SALES_IN_INTERVAL'].copy()
X = df.drop('HAS_SALES_IN_INTERVAL', axis=1)

In [ ]:
df.head(1)

,AGE,MARITAL_STATUS_Bekar,MARITAL_STATUS_Evli,TOTAL_SALES_FILE_3M,SINCE_LAST_SALES_FILE_3M,TOTAL_SALES_FILE_1Y,SINCE_LAST_SALES_FILE_1Y,TOTAL_SALES_FILE_ALL,SINCE_LAST_SALES_FILE_ALL,TOTAL_SIFIR_ARAC_3M,SINCE_LAST_SIFIR_ARAC_3M,TOTAL_SIFIR_ARAC_1Y,SINCE_LAST_SIFIR_ARAC_1Y,TOTAL_SIFIR_ARAC_ALL,SINCE_LAST_SIFIR_ARAC_ALL,HAS_SALES_IN_INTERVAL
0,46,0,1,0,0,0,0,0,0,0,0,1,0,2,0,0


In [ ]:
X = X.to_numpy()
y = y.to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=42) 

In [ ]:
train_data = (X_train, y_train)

In [ ]:
import tensorflow as tf
from tensorflow import keras

keras.backend.clear_session()
tf.random.set_seed(44)

model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(15,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, kernel_initializer='lecun_normal', activation='selu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(80, kernel_initializer='lecun_normal', activation='selu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(80, kernel_initializer='lecun_normal', activation='selu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 15)                60        
_________________________________________________________________
dense (Dense)                (None, 100)               1600      
_________________________________________________________________
batch_normalization_1 (Batch (None, 100)               400       
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 80)                8080      
_________________________________________________________________
batch_normalization_2 (Batch (None, 80)                320       
_________________________________________________________________
dropout_1 (Dropout)          (None, 80)                0

In [ ]:
opt_list = {
    "sgd": tf.keras.optimizers.SGD(learning_rate=0.01,
                            momentum=0.95,
                            nesterov=True),
    "adam": tf.keras.optimizers.Adam(learning_rate=0.01,
                            beta_1=0.9,
                            beta_2=0.999),
    "nadam": tf.keras.optimizers.Nadam(learning_rate=0.01,
                              beta_1=0.9,
                              beta_2=0.999,
                              epsilon=1e-07),
    "rmsprop":tf.keras.optimizers.RMSprop(learning_rate=0.01,
                                rho=0.9),
           }

In [ ]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                              patience=30, min_lr=0.0001)

In [ ]:
auc = tf.keras.metrics.AUC(
    num_thresholds=200, curve='ROC',
    summation_method='interpolation', name=None, dtype=None,
    thresholds=None, multi_label=False, num_labels=None, label_weights=None,
    from_logits=False
)

model.compile(loss="binary_crossentropy",
              optimizer= opt_list["adam"], # You can view the opt_list above.
              metrics=[auc],)

## 10 Epochs

In [ ]:
history = model.fit(X_train, 
                    y_train,
                    validation_split = 0.10,
                    epochs=10,
                    callbacks=reduce_lr
                    )

Epoch 1/10
4423/4423 [==============================] - 41s 9ms/step - loss: 0.0922 - auc: 0.8353 - val_loss: 0.0794 - val_auc: 0.8827
Epoch 2/10
4423/4423 [==============================] - 38s 9ms/step - loss: 0.0851 - auc: 0.8531 - val_loss: 0.0775 - val_auc: 0.8766
Epoch 3/10
4423/4423 [==============================] - 38s 9ms/step - loss: 0.0852 - auc: 0.8503 - val_loss: 0.0850 - val_auc: 0.8711
Epoch 4/10
4423/4423 [==============================] - 38s 9ms/step - loss: 0.0851 - auc: 0.8475 - val_loss: 0.0791 - val_auc: 0.8703
Epoch 5/10
4423/4423 [==============================] - 38s 9ms/step - loss: 0.0843 - auc: 0.8495 - val_loss: 0.0787 - val_auc: 0.8749
Epoch 6/10
4423/4423 [==============================] - 38s 9ms/step - loss: 0.0839 - auc: 0.8515 - val_loss: 0.0789 - val_auc: 0.8646
Epoch 7/10
4423/4423 [==============================] - 38s 9ms/step - loss: 0.0838 - auc: 0.8519 - val_loss: 0.0794 - val_auc: 0.8623
Epoch 8/10
4423/4423 [==============================] -

## 10 Epochs

In [ ]:
history = model.fit(X_train, 
                    y_train,
                    validation_split = 0.10,
                    epochs=10,
                    batch_size=512,
                    callbacks=reduce_lr
                    )

Epoch 1/10
277/277 [==============================] - 3s 9ms/step - loss: 0.0799 - auc: 0.8618 - val_loss: 0.0778 - val_auc: 0.8729
Epoch 2/10
277/277 [==============================] - 2s 9ms/step - loss: 0.0790 - auc: 0.8643 - val_loss: 0.0772 - val_auc: 0.8759
Epoch 3/10
277/277 [==============================] - 3s 10ms/step - loss: 0.0783 - auc: 0.8625 - val_loss: 0.0776 - val_auc: 0.8678
Epoch 4/10
277/277 [==============================] - 2s 9ms/step - loss: 0.0785 - auc: 0.8654 - val_loss: 0.0762 - val_auc: 0.8764
Epoch 5/10
277/277 [==============================] - 3s 9ms/step - loss: 0.0788 - auc: 0.8602 - val_loss: 0.0768 - val_auc: 0.8713
Epoch 6/10
277/277 [==============================] - 3s 9ms/step - loss: 0.0783 - auc: 0.8656 - val_loss: 0.0765 - val_auc: 0.8770
Epoch 7/10
277/277 [==============================] - 2s 9ms/step - loss: 0.0787 - auc: 0.8640 - val_loss: 0.0756 - val_auc: 0.8725
Epoch 8/10
277/277 [==============================] - 3s 9ms/step - loss: 0

## 50 epochs

In [ ]:
history = model.fit(X_train, 
                    y_train,
                    validation_split = 0.10,
                    epochs=50,
                    batch_size=512,
                    callbacks=reduce_lr
                    )

Epoch 1/50
277/277 [==============================] - 3s 10ms/step - loss: 0.0784 - auc: 0.8657 - val_loss: 0.0764 - val_auc: 0.8803
Epoch 2/50
277/277 [==============================] - 2s 9ms/step - loss: 0.0787 - auc: 0.8665 - val_loss: 0.0763 - val_auc: 0.8759
Epoch 3/50
277/277 [==============================] - 3s 9ms/step - loss: 0.0787 - auc: 0.8640 - val_loss: 0.0760 - val_auc: 0.8740
Epoch 4/50
277/277 [==============================] - 2s 9ms/step - loss: 0.0787 - auc: 0.8650 - val_loss: 0.0761 - val_auc: 0.8776
Epoch 5/50
277/277 [==============================] - 2s 9ms/step - loss: 0.0784 - auc: 0.8655 - val_loss: 0.0760 - val_auc: 0.8736
Epoch 6/50
277/277 [==============================] - 3s 9ms/step - loss: 0.0784 - auc: 0.8648 - val_loss: 0.0763 - val_auc: 0.8791
Epoch 7/50
277/277 [==============================] - 3s 9ms/step - loss: 0.0788 - auc: 0.8664 - val_loss: 0.0754 - val_auc: 0.8731
Epoch 8/50
277/277 [==============================] - 3s 9ms/step - loss: 0

# Neural Network Prediction

In [ ]:
from sklearn.metrics import roc_curve,roc_auc_score,auc

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_scores = y_pred[:,0]

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)

In [ ]:
roc_auc

0.8727799777065878

# Neural Network Prediction for 2021_06

In [ ]:
df_test = pd.read_csv('drive/MyDrive/DF_FINAL_MERGED_wSIFIR_2021_6.csv')

In [ ]:
df_test.drop(['Unnamed: 0', 'CUSTOMER_ID'], axis=1, inplace=True)
# y = df['HAS_SALES_IN_INTERVAL'].copy()
# X = df.drop('HAS_SALES_IN_INTERVAL', axis=1)

In [ ]:
df_test.head(1)

,BASE_CUSTOMER_ID,AGE,MARITAL_STATUS_Bekar,MARITAL_STATUS_Evli,TOTAL_SALES_FILE_3M,SINCE_LAST_SALES_FILE_3M,TOTAL_SALES_FILE_1Y,SINCE_LAST_SALES_FILE_1Y,TOTAL_SALES_FILE_ALL,SINCE_LAST_SALES_FILE_ALL,TOTAL_SIFIR_ARAC_3M,SINCE_LAST_SIFIR_ARAC_3M,TOTAL_SIFIR_ARAC_1Y,SINCE_LAST_SIFIR_ARAC_1Y,TOTAL_SIFIR_ARAC_ALL,SINCE_LAST_SIFIR_ARAC_ALL
0,158891,46,0,1,0,0,0,0,0,0,0,0,1,0,2,0


In [ ]:
from sklearn.metrics import roc_curve,roc_auc_score,auc

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_scores = y_pred[:,0]

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)

In [ ]:
roc_auc

0.8727799777065878

In [ ]:
sample_data = pd.read_csv('/content/sample_submission.csv.zip')

In [ ]:
sample_data.head(3)

,Id,Expected
0,168254,NaN
1,124747,NaN
2,135493,NaN


In [ ]:
df_results = pd.DataFrame(['Id', 'Expected'])

for index, row in sample_data.iterrows():
  base_id = row.loc['Id']
  df_test_w_id = df_test[ df_test['BASE_CUSTOMER_ID']== base_id ]
  test_data = df_test_w_id.iloc[0].drop('BASE_CUSTOMER_ID')

  test_data = np.expand_dims(test_data, axis=0)
  # display(test_data)

  y_pred = model.predict(test_data)
  # display(y_pred)
  y_scores = y_pred[:, 0]

  sample_data.loc[index, 'Expected'] = y_scores[0]


In [ ]:
sample_data.to_csv('drive/MyDrive/sample_submission_DNN_08714_2021_06.csv')